# Julia language introduction

## Why Julia
https://julialang.org/blog/2012/02/why-we-created-julia/

We want a language that's **open source**, with a liberal license. We want the speed of **C,** with the dynamism of **Ruby**. We want a language that's homoiconic, with true macros like **Lisp**, but with obvious, familiar mathematical notation like **Matlab**. We want something as usable for general programming as **Python**, as easy for statistics as **R,** as natural for string processing as **Perl**, as powerful for linear algebra as Matlab, as good at gluing programs together as the **shell**. Something that is dirt simple to learn, yet keeps the most serious hackers happy. We want it interactive and we want it compiled.

## Install
https://julialang.org

## REPL (read-eval-print loop)   
- The Julian mode   
- Help mode   
- Shell mode   

## Package

- add  
- using     

## Editor
- jupyter-notebook
- txt   
- vim   
- vscode https://code.visualstudio.com

## Scope of variables
The scope of a variable is the region of code within which a variable is visiable.   
Global & Local

In [1]:
for i = 1:10
    z = i
end
@show z

UndefVarError: UndefVarError: z not defined

In [10]:
for i = 1:2
    z = i
    for j = 1:2
        z = i*j
        @show z
    end
    @show z
end
@show z

z = 1
z = 2
z = 2
z = 2
z = 4
z = 4
z = 4


4

In [11]:
for i = 1:10
    global z
    z = i
end
@show z;

z = 10


In [12]:
for i = 1:10
    y = i
    global y
end
@show y;

y = 10


In [14]:
x, y = 1, 2
function foo()
    x = 2 # assignment introduces a new local variable
    return x + y
end
nothing

In [16]:
@show foo()
@show x;

foo() = 4
x = 1


In [18]:
x = 1
function foobar()
    global x = 2
end
@show foobar();
@show x;

foobar() = 2
x = 2


In [20]:
x, y = 1, 2
function baz()
    x = 2 # introduces a new local
    function bar()
        x = 10 # modifies the parent's x
        return x + y # y is global
    end
    return bar() + x
end;
@show baz()
@show (x,y)
nothing

baz() = 22
(x, y) = (1, 2)


In [21]:
x, y = 1, 2
function bar()
    x = 10 # local, no longer a closure variable
    return x + y
end
function quz()
    x = 2 # local
    return bar() + x # 12 + 2 (x is not modified)
end
@show quz()
@show (x,y)
nothing

quz() = 14
(x, y) = (1, 2)


## Type stability
Type stability is the idea that there is only **one** possible type which can be outputtted from a method. For example, the reasonable type to output from `*(::Float64,::Float64)` is a `Float64`

In [22]:
?@code_native

```
@code_native
```

Evaluates the arguments to the function or macro call, determines their types, and calls [`code_native`](@ref) on the resulting expression.

Set the optional keyword argument `debuginfo` by putting it before the function call, like this:

```
@code_native debuginfo=:default f(x)
```

`debuginfo` may be one of `:source` (default) or `:none`, to specify the verbosity of code comments.


In [23]:
function f(a,b)
    return 2a+b
end
@code_native f(2.0, 3.0)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[23]:2 within `f'
; │┌ @ promotion.jl:312 within `*' @ In[23]:2
	vaddsd	%xmm0, %xmm0, %xmm0
; │└
; │┌ @ float.jl:401 within `+'
	vaddsd	%xmm1, %xmm0, %xmm0
; │└
	retq
	nopl	(%rax)
; └


In [60]:
@code_llvm f(2.0, 3.0)


;  @ In[23]:2 within `f'
define double @julia_f_17638(double, double) {
top:
; ┌ @ promotion.jl:312 within `*' @ float.jl:405
   %2 = fmul double %0, 2.000000e+00
; └
; ┌ @ float.jl:401 within `+'
   %3 = fadd double %2, %1
; └
  ret double %3
}


In [61]:
@code_llvm f(2.0, 3)


;  @ In[23]:2 within `f'
define double @julia_f_17772(double, i64) {
top:
; ┌ @ promotion.jl:312 within `*' @ float.jl:405
   %2 = fmul double %0, 2.000000e+00
; └
; ┌ @ promotion.jl:311 within `+'
; │┌ @ promotion.jl:282 within `promote'
; ││┌ @ promotion.jl:259 within `_promote'
; │││┌ @ number.jl:7 within `convert'
; ││││┌ @ float.jl:60 within `Float64'
       %3 = sitofp i64 %1 to double
; │└└└└
; │ @ promotion.jl:311 within `+' @ float.jl:401
   %4 = fadd double %2, %3
; └
  ret double %4
}


## The REPL/Global Scope Does Not Allow Type Specificity

In [24]:
function linearcombo()
  return 2a+b
end
nothing

In [64]:
a, b = 1.0, 2.0
@show linearcombo()
@code_llvm linearcombo()

linearcombo() = 4.0

;  @ In[24]:2 within `linearcombo'
define nonnull %jl_value_t addrspace(10)* @julia_linearcombo_17640() {
top:
  %0 = alloca %jl_value_t addrspace(10)*, i32 2
  %gcframe = alloca %jl_value_t addrspace(10)*, i32 4
  %1 = bitcast %jl_value_t addrspace(10)** %gcframe to i8*
  call void @llvm.memset.p0i8.i32(i8* %1, i8 0, i32 32, i32 0, i1 false)
  %2 = call %jl_value_t*** inttoptr (i64 4500222000 to %jl_value_t*** ()*)() #4
  %3 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 0
  %4 = bitcast %jl_value_t addrspace(10)** %3 to i64*
  store i64 4, i64* %4
  %5 = getelementptr %jl_value_t**, %jl_value_t*** %2, i32 0
  %6 = load %jl_value_t**, %jl_value_t*** %5
  %7 = getelementptr %jl_value_t addrspace(10)*, %jl_value_t addrspace(10)** %gcframe, i32 1
  %8 = bitcast %jl_value_t addrspace(10)** %7 to %jl_value_t***
  store %jl_value_t** %6, %jl_value_t*** %8
  %9 = bitcast %jl_value_t*** %5 to %jl_value_t addrspace(10)***
  store %jl_

In [25]:
@code_native linearcombo()

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[24]:2 within `linearcombo'
	pushq	%rbp
	movq	%rsp, %rbp
	pushq	%r15
	pushq	%r14
	pushq	%r13
	pushq	%r12
	pushq	%rbx
	andq	$-32, %rsp
	subq	$96, %rsp
	movabsq	$jl_system_image_data, %r14
	vxorps	%xmm0, %xmm0, %xmm0
	vmovaps	%ymm0, 32(%rsp)
	leaq	-79875808(%r14), %rax
	vzeroupper
	callq	*%rax
	movq	%rax, %r15
	movq	$4, 32(%rsp)
	movq	(%r15), %rax
	movq	%rax, 40(%rsp)
	leaq	32(%rsp), %rax
	movq	%rax, (%r15)
	movabsq	$5057822296, %rbx       ## imm = 0x12D783E58
	movq	(%rbx), %rax
	testq	%rax, %rax
	jne	L124
	leaq	-25150048(%r14), %rsi
	movabsq	$jl_get_binding_or_error, %rax
	movq	%r14, %rdi
	callq	*%rax
	movq	%rax, (%rbx)
L124:
	movq	8(%rax), %rax
	testq	%rax, %rax
	je	L323
	movq	%rax, 48(%rsp)
; │ @ In[24]:2 within `linearcombo'
	movabsq	$4554506400, %rcx       ## imm = 0x10F7840A0
	movq	%rcx, 16(%rsp)
	movq	%rax, 24(%rsp)
	movabsq	$jl_apply_generic, %r12
	movabsq	$jl_system_image_data, %rdi
	leaq	16(%rsp), %rsi
	movl	$2, %edx
	ca

In [66]:
const c = 1.0
const d = 2.0
function linearcombo2()
  return 2c+d
end
@code_llvm linearcombo2()


;  @ In[66]:4 within `linearcombo2'
define double @julia_linearcombo2_17791() {
top:
  ret double 4.000000e+00
}


In [67]:
@code_native linearcombo2()

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[66]:4 within `linearcombo2'
	movabsq	$5057954320, %rax       ## imm = 0x12D7A4210
	vmovsd	(%rax), %xmm0           ## xmm0 = mem[0],zero
	retq
	nop
; └


In [94]:
function linearcombo3()
    a, b = 2.0, 3.0
    function linearcombo4()
        return 2a+b
    end
    return linearcombo4()
end
@code_llvm linearcombo3()


;  @ In[94]:2 within `linearcombo3'
define double @julia_linearcombo3_17962() {
top:
;  @ In[94]:6 within `linearcombo3'
  ret double 7.000000e+00
}


In [95]:
linearcombo3()

7.0

In [96]:
@code_native linearcombo3()

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[94]:2 within `linearcombo3'
	movabsq	$5058162968, %rax       ## imm = 0x12D7D7118
	vmovsd	(%rax), %xmm0           ## xmm0 = mem[0],zero
	retq
	nop
; └


In [101]:
@time linearcombo()
@time linearcombo2()
@time linearcombo3()
nothing

  0.000005 seconds (6 allocations: 192 bytes)
  0.000002 seconds (4 allocations: 160 bytes)
  0.000002 seconds (4 allocations: 160 bytes)


In [98]:
function g(x)
    for i = 1:10000
        x /= 2
    end
    return x
end
nothing

In [71]:
@time g(10)
@time g(10.0)
nothing

  0.000043 seconds (5 allocations: 176 bytes)
  0.000018 seconds (5 allocations: 176 bytes)


In [74]:
@code_native g(10)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[68]:2 within `g'
	pushq	%rax
	movb	$2, %cl
	movl	$9999, %eax             ## imm = 0x270F
	movabsq	$5057954624, %rdx       ## imm = 0x12D7A4340
	vmovsd	(%rdx), %xmm1           ## xmm1 = mem[0],zero
	andb	$3, %cl
; │ @ In[68]:3 within `g'
	cmpb	$1, %cl
	jne	L51
	jmp	L80
L32:
	vmovq	%xmm0, %rdi
	addq	$-1, %rax
	movb	$1, %cl
	andb	$3, %cl
	cmpb	$1, %cl
	je	L80
L51:
	cmpb	$2, %cl
	jne	L96
; │┌ @ int.jl:59 within `/'
; ││┌ @ float.jl:277 within `float'
; │││┌ @ float.jl:262 within `AbstractFloat'
; ││││┌ @ float.jl:60 within `Float64'
	vcvtsi2sdq	%rdi, %xmm2, %xmm0
; │└└└└
; │┌ @ float.jl:407 within `/'
	vmulsd	%xmm1, %xmm0, %xmm0
; │└
; │┌ @ range.jl:597 within `iterate'
; ││┌ @ promotion.jl:401 within `=='
	testq	%rax, %rax
; │└└
	jne	L32
	jmp	L94
	nopl	(%rax,%rax)
; │┌ @ promotion.jl:314 within `/' @ float.jl:407
L80:
	vmovq	%rdi, %xmm0
; │└
; │┌ @ int.jl:59 within `/' @ float.jl:407
	vmulsd	%xmm1, %xmm0, %xmm0
; │└
; │┌ @ range.j

In [75]:
@code_native g(10.0)

	.section	__TEXT,__text,regular,pure_instructions
; ┌ @ In[68]:2 within `g'
	movl	$10000, %eax            ## imm = 0x2710
	movabsq	$5057956136, %rcx       ## imm = 0x12D7A4928
	vmovsd	(%rcx), %xmm1           ## xmm1 = mem[0],zero
	nopw	%cs:(%rax,%rax)
; │ @ In[68]:3 within `g'
; │┌ @ promotion.jl:314 within `/' @ float.jl:407
L32:
	vmulsd	%xmm1, %xmm0, %xmm0
; │└
; │┌ @ range.jl:597 within `iterate'
; ││┌ @ promotion.jl:401 within `=='
	addq	$-1, %rax
; │└└
	jne	L32
; │ @ In[68]:5 within `g'
	retq
	nopl	(%rax,%rax)
; └


In [72]:
@code_warntype g(10)

Variables
  #self#::Core.Compiler.Const(g, false)
  x@_2::Int64
  @_3::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  x@_5::Union{Float64, Int64}

Body::Float64
1 ─       (x@_5 = x@_2)
│   %2  = (1:10000)::Core.Compiler.Const(1:10000, false)
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3::Core.Compiler.Const((1, 1), false) === nothing)::Core.Compiler.Const(false, false)
│   %5  = Base.not_int(%4)::Core.Compiler.Const(true, false)
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (x@_5 = x@_5 / 2)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return x@_5::Float64


In [73]:
@code_warntype g(10.0)

Variables
  #self#::Core.Compiler.Const(g, false)
  x@_2::Float64
  @_3::Union{Nothing, Tuple{Int64,Int64}}
  i::Int64
  x@_5::Float64

Body::Float64
1 ─       (x@_5 = x@_2)
│   %2  = (1:10000)::Core.Compiler.Const(1:10000, false)
│         (@_3 = Base.iterate(%2))
│   %4  = (@_3::Core.Compiler.Const((1, 1), false) === nothing)::Core.Compiler.Const(false, false)
│   %5  = Base.not_int(%4)::Core.Compiler.Const(true, false)
└──       goto #4 if not %5
2 ┄ %7  = @_3::Tuple{Int64,Int64}::Tuple{Int64,Int64}
│         (i = Core.getfield(%7, 1))
│   %9  = Core.getfield(%7, 2)::Int64
│         (x@_5 = x@_5 / 2)
│         (@_3 = Base.iterate(%2, %9))
│   %12 = (@_3 === nothing)::Bool
│   %13 = Base.not_int(%12)::Bool
└──       goto #4 if not %13
3 ─       goto #2
4 ┄       return x@_5


In [1]:
2^-5

0.03125